In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

In [ ]:
# read the data
#flights 
df= pd.read_csv('../input/flight-delays/flights.csv')
#airports
df1 = pd.read_csv('../input/flight-delays/airports.csv')
#airlines
df2= pd.read_csv('../input/flight-delays/airlines.csv')

# shape and data types of the data
print(df.shape)
print(df.dtypes)

# select numeric columns
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print("numeric_cols :",numeric_cols)

# select non numeric columns
df_non_numeric = df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print("non_numeric_cols :",non_numeric_cols)

In [ ]:
df.head()

In [ ]:
#Données manquantes.
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

In [ ]:
df.isnull().sum()

In [ ]:
#VitesseMoyenneAvion
df['VitesseMoyenneAvion']=df['DISTANCE']*60/df['AIR_TIME']

In [ ]:
df['AIR_SYSTEM_DELAY'].fillna(int(0), inplace=True)
df['WEATHER_DELAY'].fillna(int(0), inplace=True)
df['LATE_AIRCRAFT_DELAY'].fillna(int(0), inplace=True)
df['SECURITY_DELAY'].fillna(int(0), inplace=True)
df['AIRLINE_DELAY'].fillna(int(0), inplace=True)

In [ ]:
#ConsomSuppCarb
df['ConsomSuppCarb']=(df['AIR_SYSTEM_DELAY']+df['WEATHER_DELAY']+df['LATE_AIRCRAFT_DELAY']+df['SECURITY_DELAY']+df['AIRLINE_DELAY'])*60*5.5

In [ ]:
#Moyenne TaxiOut par aéroport / Jointure avec la table Flights
Moyenne_TaxiOut_par_aéroport=df[['ORIGIN_AIRPORT','TAXI_OUT']].groupby('ORIGIN_AIRPORT')['TAXI_OUT'].mean()
df=df.join(Moyenne_TaxiOut_par_aéroport, on='ORIGIN_AIRPORT', rsuffix='_Moyenne')

In [ ]:
#Moyenne TaxiIN par aéroport / Jointure avec la table Flights
Moyenne_TaxiIN_par_aéroport=df[['DESTINATION_AIRPORT','TAXI_IN']].groupby('DESTINATION_AIRPORT')['TAXI_IN'].mean()
df=df.join(Moyenne_TaxiIN_par_aéroport, on='DESTINATION_AIRPORT', rsuffix='_Moyenne')

In [ ]:
df['TaxiInSupp']=df['TAXI_IN']-df['TAXI_IN_Moyenne']
df['TaxiOutSupp']=df['TAXI_OUT']-df['TAXI_OUT_Moyenne']

In [ ]:
import gc
del df['TAXI_IN_Moyenne']
gc.collect()

In [ ]:
import gc
del df['TAXI_OUT_Moyenne']
gc.collect()

In [ ]:
df['SCHEDULED_ARRIVAL'].fillna(0, inplace=True)
df['ARRIVAL_TIME'].fillna(0, inplace=True)

In [ ]:
df.loc[df['CANCELLED'] == 1, 'Statut'] = 'Annulé' 
df.loc[df['DIVERTED'] == 1, 'Statut'] = 'Dérouté'
df.loc[(df['ARRIVAL_TIME'].astype(int)-df['SCHEDULED_ARRIVAL'].astype(int)<=0) & (df['CANCELLED'] != 1) & (df['DIVERTED'] != 1) , 'Statut'] = 'A temps'
df.loc[(df['ARRIVAL_TIME'].astype(int)-df['SCHEDULED_ARRIVAL'].astype(int)>0) & (df['CANCELLED'] != 1) & (df['DIVERTED'] != 1) , 'Statut'] = 'Retardé'

In [ ]:
df['CANCELLATION_REASON'].fillna("E", inplace=True)
df.loc[df['CANCELLATION_REASON'] == "A", 'FactAnnulation'] = 'Airline/Carrier' 
df.loc[df['CANCELLATION_REASON'] == "B", 'FactAnnulation'] = 'Weather'
df.loc[df['CANCELLATION_REASON'] == "C", 'FactAnnulation'] = 'National Air System'
df.loc[df['CANCELLATION_REASON'] == "D", 'FactAnnulation'] = 'Security'
df.loc[df['CANCELLATION_REASON'] == "E", 'FactAnnulation'] = 'Aucun'

In [ ]:
df['DATE']=df['YEAR'].astype(str)+"-"+df['MONTH'].astype(str)+"-"+df['DAY'].astype(str)

In [ ]:
df.head()

In [ ]:
# La Comparaison des moyennes de durées des différents facteurs de retard de vols pendant l'année.
plt.figure(figsize=(30,4))

a=df.groupby(by='DATE')['AIR_SYSTEM_DELAY'].agg('mean')
b=df.groupby(by='DATE')['WEATHER_DELAY'].agg('mean')
c=df.groupby(by='DATE')['LATE_AIRCRAFT_DELAY'].agg('mean')
d=df.groupby(by='DATE')['AIRLINE_DELAY'].agg('mean')
e=df.groupby(by='DATE')['SECURITY_DELAY'].agg('mean')

sns.lineplot(data=a, color='#CD5C5C', label="AIR SYSTEM DELAY").set_title("Fluctuation des durées de retards par jour pour chaque facteur")
sns.lineplot(data=b, color='green', label="WEATHER DELAY")
sns.lineplot(data=c, color='blue', label="LATE AIRCRAFT DELAY")
sns.lineplot(data=d, color='black', label="AIRLINE DELAY")
sns.lineplot(data=e, color='purple', label="SECURITY DELAY")



plt.ylabel('Retard en minutes') 

In [ ]:
# La Comparaison des moyennes des durées des différents facteurs de retard de vols par mois.
plt.figure(figsize=(30,4))

a=df.groupby(by='MONTH')['AIR_SYSTEM_DELAY'].agg('mean')
b=df.groupby(by='MONTH')['WEATHER_DELAY'].agg('mean')
c=df.groupby(by='MONTH')['LATE_AIRCRAFT_DELAY'].agg('mean')
d=df.groupby(by='MONTH')['AIRLINE_DELAY'].agg('mean')
e=df.groupby(by='MONTH')['SECURITY_DELAY'].agg('mean')

sns.lineplot(data=a, color='#CD5C5C', label="AIR SYSTEM DELAY").set_title("Fluctuation des durées de retards par mois pour chaque facteur")
sns.lineplot(data=b, color='green', label="WEATHER DELAY")
sns.lineplot(data=c, color='blue', label="LATE AIRCRAFT DELAY")
sns.lineplot(data=d, color='black', label="AIRLINE DELAY")
sns.lineplot(data=e, color='purple', label="SECURITY DELAY")



plt.ylabel('Retard en minutes') 

In [ ]:
Pourcentge_Retard_AirSYSTEM=df['AIR_SYSTEM_DELAY'].sum()*100/(df['AIR_SYSTEM_DELAY'].sum()+df['LATE_AIRCRAFT_DELAY'].sum()+df['AIRLINE_DELAY'].sum()+df['SECURITY_DELAY'].sum()+df['WEATHER_DELAY'].sum())

In [ ]:
print(round(Pourcentge_Retard_AirSYSTEM),"%")

In [ ]:
Pourcentge_Retard_AirLine=df['AIRLINE_DELAY'].sum()*100/(df['AIR_SYSTEM_DELAY'].sum()+df['LATE_AIRCRAFT_DELAY'].sum()+df['AIRLINE_DELAY'].sum()+df['SECURITY_DELAY'].sum()+df['WEATHER_DELAY'].sum())
print(round(Pourcentge_Retard_AirLine),"%")

In [ ]:
Pourcentge_Retard_AirCraft=df['LATE_AIRCRAFT_DELAY'].sum()*100/(df['AIR_SYSTEM_DELAY'].sum()+df['LATE_AIRCRAFT_DELAY'].sum()+df['AIRLINE_DELAY'].sum()+df['SECURITY_DELAY'].sum()+df['WEATHER_DELAY'].sum())
print(round(Pourcentge_Retard_AirCraft),"%")

In [ ]:
Pourcentge_Retard_SECURITY=df['SECURITY_DELAY'].sum()*100/(df['AIR_SYSTEM_DELAY'].sum()+df['LATE_AIRCRAFT_DELAY'].sum()+df['AIRLINE_DELAY'].sum()+df['SECURITY_DELAY'].sum()+df['WEATHER_DELAY'].sum())
print(Pourcentge_Retard_SECURITY,"%")

In [ ]:
Pourcentge_Retard_Weather=df['WEATHER_DELAY'].sum()*100/(df['AIR_SYSTEM_DELAY'].sum()+df['LATE_AIRCRAFT_DELAY'].sum()+df['AIRLINE_DELAY'].sum()+df['SECURITY_DELAY'].sum()+df['WEATHER_DELAY'].sum())
print(round(Pourcentge_Retard_Weather),"%")

On remarque que les avions représentent le facteur (Late Aircraft Delay) qui cause le plus de retard. En effet, la courbe de ce facteur est celle qui prend la valeur maximale presque chaque jour/chaque mois.
Ensuite on a le deuxième facteur du retard le plus pesant est celui causé par les compagnies aériennes, puis les retards causés par les systèmes aériens.
Quand aux retards causés par les conditions météorologiques et la sécurité sont minimes comparés aux autres facteurs.

On remarque de même que les retards augmentent et atteignent leurs maximums pendant le mois 6 et le mois 12. Ce qui est tout a fait logique, puisque le mois 6 correspond aux vacances d'été et le mois 12 aux vacances de la fin d'années, la période où les gens voyagent le plus, donc la demande augmente,le service subissant une grande pression, il va se détorioer en contre partie.

In [ ]:
# La Comparaison des moyenne des différents causes d'annulations d'avions par mois.
plt.figure(figsize=(30,4))

sns.barplot(x=df["FactAnnulation"].where(df["FactAnnulation"]!="Aucun").index,y=df["FactAnnulation"].where(df["FactAnnulation"]!="Aucun"), label="Facteurs d'annulation")




On remarque que la majorité des cas d'annulation de vols sont causés majoritairement par des facteurs de sécurité.
La météo étant le facteur qui cause le moins l'annulation des vols.

On conclue que dans les deux cas (annulations et retards de vols), cela revient principalement à des problèmes techniques qui peuvent éventuellement être contrôlés en améliorant la qualité du service offert.

**ANALYSE SUR LES AEROPORTS**

In [ ]:
DelayDepart_par_aéroport=df[['ORIGIN_AIRPORT','DEPARTURE_DELAY']].groupby('ORIGIN_AIRPORT')['DEPARTURE_DELAY'].mean()
DelayArrive_par_aéroport=df[['DESTINATION_AIRPORT','ARRIVAL_DELAY']].groupby('DESTINATION_AIRPORT')['ARRIVAL_DELAY'].mean()

In [ ]:
DelayDepart_par_aéroport.isnull().sum()
DelayArrive_par_aéroport.isnull().sum()

In [ ]:
res = pd.concat([DelayDepart_par_aéroport, DelayArrive_par_aéroport],axis=1)

In [ ]:
res['MoyDelay']=round((res['DEPARTURE_DELAY']+res['ARRIVAL_DELAY'])/2).sort_values(ascending=False)
#on n'a pas utilisé(heure arrivée - heure proogrammée) car il y aura l'influence du retard du à l'avion

In [ ]:
res.rename_axis('Aeroport', axis = 'rows') 

In [ ]:
l=res.index.values
len(l)

In [ ]:
index_non_digit=[ x for x in l if str(x).isdigit() ]

In [ ]:
res = res.drop(index_non_digit) 

In [ ]:
res

In [ ]:
sns.barplot(x=res['MoyDelay'].sort_values(ascending=False).index, y=res['MoyDelay'].sort_values(ascending=False), label='Retard moyen par aéroport')
#Moyenne retard par aéroport

In [ ]:
res.sort_values('MoyDelay', ascending=False, inplace=True)

In [ ]:
res

In [ ]:
res.head(10)

TOP 10 des aéroports qui ont fait l'objet de plus de retards: 

    * Wilmington Airport
    
    * St. Cloud Regional Airport
    
    * Guam International Airport
    
    * Southwest Oregon Regional Airport (North Bend Municipal)
    
    * Aspen-Pitkin County Airport
    
    * Jack Brooks Regional Airport (Southeast Texas Regional)
    
    * Trenton Mercer Airport
    
    * Plattsburgh International Airport
    
    * Gustavus Airport
#     
    * Martha's Vineyard Airport


In [ ]:
res.tail(10)

In [ ]:
df.columns

**ANALYSE SUR LES Compagnies Aériennes**

In [ ]:
#Retard Total pour chaque vol
df['RetardTotal']=df['AIR_SYSTEM_DELAY']+df['WEATHER_DELAY']+df['LATE_AIRCRAFT_DELAY']+df['SECURITY_DELAY']+df['AIRLINE_DELAY']

In [ ]:
#Retard Moyen par Compagnies aériennes
DelayMOY_par_AIRLINE=df[['AIRLINE','AIRLINE_DELAY']].groupby('AIRLINE')['AIRLINE_DELAY'].mean()

In [ ]:
DelayMOY_par_AIRLINE.sort_values(ascending=False, inplace=True)

In [ ]:
DelayMOY_par_AIRLINE

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
DelayMOY_par_AIRLINE.plot(kind="bar")

Classement DES COMPAGNIES AEERIENNES Faisant des retards.

    * Spirit Air Lines

    * Frontier Airlines Inc.

    * JetBlue Airways

    * United Air Lines Inc.

    * American Eagle Airlines Inc.

    * Atlantic Southeast Airlines

    * Skywest Airlines Inc.

    * American Airlines Inc.

    * Virgin America

    * Southwest Airlines Co.

    * US Airways Inc.

    * Delta Air Lines Inc.

    * Alaska Airlines Inc.

    * Hawaiian Airlines Inc.

In [ ]:
#ANALYSE DES AVIONS DE LA COMPAGNIE "Spirit Air Lines" - UA.
P=df[df['AIRLINE']=="UA"]

In [ ]:
#Retard Moyen par avion de la Compagnie aérienne "UA".
DelayMOY_par_AVION_UA=P[['TAIL_NUMBER','LATE_AIRCRAFT_DELAY']].groupby('TAIL_NUMBER')['LATE_AIRCRAFT_DELAY'].mean()

In [ ]:
DelayMOY_par_AVION_UA.sort_values(ascending=False, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(32,15))
DelayMOY_par_AVION_UA.plot(kind="bar")

In [ ]:
print("Nombre d'avions de cette compagnie aérienne =", DelayMOY_par_AVION_UA.count(),"avions.")

In [ ]:
TOP20 = DelayMOY_par_AVION_UA.head(20)

In [ ]:
TOP20

In [ ]:
i=1
for x in TOP20.index:
  #prc: pourcentage par rapport à la valeur max
  prc=round(100*TOP20[i-1]/TOP20[0],2)
  print("Retard Avion","\"" ,x,"\"","% au MAX =",prc,"%")
  i+=1

On remarque que le retard de la compagnie "Spirit Air Lines" est dû en grande partie à cause de 3 avions: N37018 - N79011 - N128UA.
Est-ce que ces avions sont vraiment les causes du retard? ou bien c'est juste par malchance que dans les vols qu'ils ont effectué, les autres facteurs ont causé ces retards?

In [ ]:
AviN37=df[df['TAIL_NUMBER']=="N37018"]
Pourcentge_Retard_N37018=AviN37['LATE_AIRCRAFT_DELAY'].sum()*100/(AviN37['AIR_SYSTEM_DELAY'].sum()+AviN37['LATE_AIRCRAFT_DELAY'].sum()+AviN37['AIRLINE_DELAY'].sum()+AviN37['SECURITY_DELAY'].sum()+AviN37['WEATHER_DELAY'].sum())
print("Pourcentage du retard causé par l'avion N37018 lors des différents vols qu'elle a effectué, % aux autres facteurs de retards : ",round(Pourcentge_Retard_N37018),"%")

In [ ]:
AviN79=df[df['TAIL_NUMBER']=="N79011"]
Pourcentge_Retard_AviN79=AviN79['LATE_AIRCRAFT_DELAY'].sum()*100/(AviN79['AIR_SYSTEM_DELAY'].sum()+AviN79['LATE_AIRCRAFT_DELAY'].sum()+AviN79['AIRLINE_DELAY'].sum()+AviN79['SECURITY_DELAY'].sum()+AviN79['WEATHER_DELAY'].sum())
print("Pourcentage du retard causé par l'avion N79011 lors des différents vols qu'elle a effectué, % aux autres facteurs de retards : ",round(Pourcentge_Retard_AviN79),"%")

In [ ]:
AviN12=df[df['TAIL_NUMBER']=="N128UA"]
Pourcentge_Retard_AviN12=AviN12['LATE_AIRCRAFT_DELAY'].sum()*100/(AviN12['AIR_SYSTEM_DELAY'].sum()+AviN12['LATE_AIRCRAFT_DELAY'].sum()+AviN12['AIRLINE_DELAY'].sum()+AviN12['SECURITY_DELAY'].sum()+AviN12['WEATHER_DELAY'].sum())
print("Pourcentage du retard causé par l'avion N128UA lors des différents vols qu'elle a effectué, % aux autres facteurs de retards : ",round(Pourcentge_Retard_AviN12),"%")

On conclue que ces trois avions contribuent grandement aux retards des vols, donc ils sont l'uns des facteurs principales causant le retards des vols qu'ils effectuent. Surtout l'avion N37018 avec 67% par rapport aux autres facteurs. Elle pourrait représenter quelques défections,ou elle rencontre des problème au niveau de sa gestion. 

**ANALYSE GENERALE SUR LES AVIONS**

In [ ]:
#Moyennes des retards effectués par chaque avion pendant l'année 2015 ordonnées descendant
DelayMOY_par_AVION=df[['TAIL_NUMBER','LATE_AIRCRAFT_DELAY']].groupby('TAIL_NUMBER')['LATE_AIRCRAFT_DELAY'].mean()
DelayMOY_par_AVION.sort_values(ascending=False, inplace=True)
DelayMOY_par_AVION

In [ ]:
#A quelles compagnies aériennes appartiennent ces avions?
TOP20 = DelayMOY_par_AVION.head(20)
TOP20


In [ ]:
z=df[['AIRLINE','TAIL_NUMBER']].loc[df['TAIL_NUMBER'].isin(TOP20.index)]

In [ ]:
z.drop_duplicates()

In [ ]:
#Les compagnies auquelles appartiennent ces avions dans le TOP20 avec le pourcentage.
round(z["AIRLINE"].value_counts(normalize=True) * 100,2)


Les avions de la compagnie "Spirit Air Lines" sont ceux qui effectuent le plus de retards. En effet, 62% dans le TOP 20 des avions retardataires appartiennent à cette compagnie. (à éviter pour vos prochains voyages.)

In [ ]:
#TOP DES AVIONS POUCTUELS
TOP20_Ponct = DelayMOY_par_AVION.tail(20)

In [ ]:
TOP20_Ponct

In [ ]:
q=df[['AIRLINE','TAIL_NUMBER']].loc[df['TAIL_NUMBER'].isin(TOP20_Ponct.index)]
q.drop_duplicates()
#Les compagnies auquelles appartiennent ce TOP 20 des avions ponctuels avec le pourcentage.
round(q["AIRLINE"].value_counts(normalize=True) * 100,2)

Delta Air Lines Inc. est la compagnie aérienne dont les avions sont les plus ponctuels.


In [ ]:
# % de vols en retard au départ (Heure réelle de départ – Heure de départ programmée)
# % de vols en retard à l'arrivée (Heure réelle d'arrivée– Heure d'arrivée programmée)

Vols_Retardé_Dep=(df["DEPARTURE_DELAY"].notnull() | df["DEPARTURE_DELAY"]!=0).sum()
Total_Vols=df["FLIGHT_NUMBER"].count()
Pourcentg_vols_en_retard_au_départ=Vols_Retardé/Total_Vols*100
print("Pourcentage de vols retardés au départ = ",round(Pourcentg_vols_en_retard_au_départ,2), "%")

In [ ]:
# % de vols en retard à l'arrivée (Heure réelle d'arrivée– Heure d'arrivée programmée)

Vols_Retardé_Arriv=(df["ARRIVAL_DELAY"].notnull() | df["ARRIVAL_DELAY"]!=0).sum()
Pourcentg_vols_en_retard_arrivee=Vols_Retardé_Arriv/Total_Vols*100
print("Pourcentage de vols retardés à l'arrivée = ",round(Pourcentg_vols_en_retard_arrivee,2), "%")

In [ ]:
#Comparaison des durées de retards au départ vs à l'arrivée

Duree_Ret_Dep = df["DEPARTURE_DELAY"][df["DEPARTURE_DELAY"].notnull()].sum()
Duree_Ret_Arr = df["ARRIVAL_DELAY"][df["ARRIVAL_DELAY"].notnull()].sum()
sns.barplot(["Durée Retard au Départ","Durée Retard à l'Arrivée"],[Duree_Ret_Dep,Duree_Ret_Arr])
Prct_Dep=Duree_Ret_Dep*100/(Duree_Ret_Dep+Duree_Ret_Arr)
Prct_Arr=Duree_Ret_Arr*100/(Duree_Ret_Dep+Duree_Ret_Arr)
print("Pourcentage de la durée de retard au départ",round(Prct_Dep,2)," %")
print("Pourcentage de la durée de retard au départ",round(Prct_Arr,2)," %")

Le nombre de vols retardés au départ est égal à ceux retardés à l'arrivée (le même pourcentage = 98%). Parcontre, la durée de retard total au départ observée sur tous les vols au cours de l'année 2015 est largement grand (68.08% de la durée total de retard) par rapport au retard d'arrivée (31.92% de la durée total de retard).

In [ ]:
# Adhésion aux créneaux de décollage = Nombre de vols où le créneau de décollage a été respecté*100/nombre de vols  
Vol_Creneau_Respecté=df[df['SCHEDULED_DEPARTURE']==df['DEPARTURE_TIME']]

In [ ]:
Adhésion_aux_créneaux_de_décollage=(Vol_Creneau_Respecté['FLIGHT_NUMBER'].count()*100)/Total_Vols
print('Adhésion aux créneaux de décollage',Adhésion_aux_créneaux_de_décollage, "%")

In [ ]:
# Poucentage des différents statuts des vols effectués
round(df["Statut"].value_counts(normalize=True) * 100,2)

* 63.3% des vols ont respecté le créneau programmé du vol.

* 34.89% des vols ont connu des retards.

* 1.54% des vols ont été annulés.
 
* 0.26% des vols ont dérouté.

In [ ]:
# Consommation Supplémentaire de Carburant par Avion
Consom_Supp_Par_Avion= df[['ConsomSuppCarb','TAIL_NUMBER']].groupby('TAIL_NUMBER')['ConsomSuppCarb'].sum()

In [ ]:
Consom_Supp_Par_Avion.sort_values(ascending=False, inplace=True)

In [ ]:
Consom_Supp_Par_Avion

In [ ]:
# Plot des avions qui consomment le plus de carburants supplémentaires par rapport à la moyenne.
Consom_Supp_Par_Avion.plot(kind='bar')

In [ ]:
# Plot des 20 avions qui consomment le plus de carburants supplémentaires par rapport à la moyenne.
Consom_Supp_Par_Avion.head(20).plot(kind='bar')

In [ ]:
i=1
for x in Consom_Supp_Par_Avion.index:
  #prc: pourcentage par rapport à la valeur max
  prc=round(100*Consom_Supp_Par_Avion[i-1]/Consom_Supp_Par_Avion[0],2)
  print("Consommation supp de carbuant de l'Avion","\"" ,x,"\"","% au MAX =",prc,"%")
  i+=1

On remarque qu'il y a  une grande disparité entre les avions quant à la consommation du carburant. Les avions au top de cette liste font subir leurs compagnies aériennes de grosses pertes en termes d'argent.